# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import api_keys

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path="../output_data/cities.csv"
city_df = pd.read_csv(weather_data_path)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Olinda,-8.0089,-34.8553,80.60,65,75,9.22,BR,1610665563
1,Talaya,61.3833,152.7500,-38.15,79,100,0.63,RU,1610665563
2,Iqaluit,63.7506,-68.5145,-13.00,77,20,5.75,CA,1610665564
3,Ushuaia,-54.8000,-68.3000,53.60,58,20,10.36,AR,1610665475
4,San Quintín,30.4833,-115.9500,69.69,34,0,6.55,MX,1610665564


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key = g_key)

# store longitude and Latitude
locs = city_df[['Lat','Lng']].astype(float)

# store humidity for the weights
humidity = city_df['Humidity'].astype(float)

In [9]:
# format the layout
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create heatmap layer
fig = gmaps.figure(layout = figure_layout)

heat_layer = gmaps.heatmap_layer(locs, 
                              weights = humidity, 
                              dissipating = False, 
                              max_intensity = 100, 
                              point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# New data frame with the criteria
city_df_filter = city_df.loc[(city_df['Max Temp'] >= 70) 
                             & (city_df['Max Temp'] < 80)
                             & (city_df['Wind Speed'] < 10)
                             & (city_df['Cloudiness'] == 0)]
city_df_filter.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
86,Dalby,-27.1833,151.2667,73.40,64,0,5.75,AU,1610665703
146,Kudāl,16.0333,73.6833,74.35,53,0,2.86,IN,1610665775
227,Wawa,9.9022,4.4192,76.50,18,0,5.70,NG,1610665913
343,Caravelas,-17.7125,-39.2481,77.65,81,0,7.63,BR,1610666060
363,Todos Santos,23.4500,-110.2167,72.00,64,0,4.00,MX,1610666124


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# copy the filtered DF
hotel_df = city_df_filter.copy()
hotel_df["Hotel Name"]=""
city_limit = 10

# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
results = ""
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    params = {
    "radius": 5000, # radius
    "location": f"{row['Lat']},{row['Lng']}",
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key}

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        # get hotel name and replace the coordinates with actual hotel coordinates
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# remove missing hotel names
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)

# apply city count limit
hotel_df = hotel_df.head(city_limit)

Closest hotel is Drovers Motor Inn.
------------
Closest hotel is Hotel Kulswamini Kudal,hotel in kudal.
------------
Missing field/result... skipping.
------------
Closest hotel is Mar Doce de Abrolhos.
------------
Closest hotel is Hotel California Todos Santos.
------------
Closest hotel is Bonthe Holiday Village.
------------
Missing field/result... skipping.
------------
Closest hotel is Hotel Apacheta.
------------
Missing field/result... skipping.
------------


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info)

# Display figure
fig = gmaps.figure(layout = figure_layout)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…